In [17]:
pessoas = [('Lisboa', 'LIS'),
           ('Madrid', 'MAD'),
           ('Paris', 'CDG'),
           ('Dublin', 'DUB'),
           ('Bruxelas', 'BRU'),
           ('Londres', 'LHR')]

In [18]:
pessoas[5]

('Londres', 'LHR')

In [19]:
destino = 'FCO'

In [20]:
voos = {('BRU', 'FCO'): ['15:44', '18:55', 382]}

In [21]:
voos

{('BRU', 'FCO'): ['15:44', '18:55', 382]}

<h2>Divisão de vôos por origem e destino</h2>

In [22]:
voos = {}
for linha in open('/Users/alvin/Documents/MyProjects/PythonAI/optimization_algorithms/flights.txt'):
    origem, destino, saida, chegada, preco = linha.split(',')
    voos.setdefault((origem, destino), [])
    voos[(origem, destino)].append((saida, chegada, int(preco)))

In [23]:
voos[('LIS', 'FCO')]

[('6:11', '8:31', 249),
 ('7:39', '10:24', 219),
 ('9:15', '12:03', 99),
 ('11:08', '13:07', 175),
 ('12:18', '14:56', 172),
 ('13:37', '15:08', 250),
 ('15:03', '16:42', 135),
 ('16:51', '19:09', 147),
 ('18:12', '20:17', 242),
 ('20:05', '22:06', 261)]

In [24]:
agenda = [1,2, 3,2, 7,3, 6,3, 2,4, 5,3]
len(agenda)

12

In [25]:
len(agenda) // 2

6

In [26]:
pessoas[1][1]

'MAD'

In [27]:
def imprimir_voos(agenda):
    id_voo = -1
    total_preco = 0
    for i in range(len(agenda) // 2):
        nome = pessoas[i][0]
        origem = pessoas[i][1]
        id_voo += 1
        ida = voos[(origem, destino)][agenda[id_voo]]
        total_preco += ida[2]
        id_voo += 1
        volta = voos[(destino, origem)][agenda[id_voo]]
        total_preco += volta[2]
        print('%10s%10s %5s-%5s %3s %5s-%5s %3s' % (nome, origem, ida[0], ida[1], ida[2],
                                                    volta[0], volta[1], volta[2]))
    print('Preço total: ', total_preco)

In [28]:
agenda = [1,0, 3,2, 7,1, 6,3, 2,4, 5,3]
imprimir_voos(agenda)

    Lisboa       LIS  7:39-10:24 219  6:19- 8:13 239
    Madrid       MAD 11:01-12:39 260  9:11-10:42 172
     Paris       CDG 17:07-20:04 291  8:23-11:07 143
    Dublin       DUB 15:27-17:18 151 10:33-12:03  74
  Bruxelas       BRU  9:08-12:12 364 12:20-16:34 500
   Londres       LHR 13:40-15:38 137 10:32-13:16 139
Preço total:  2689


<h1>Fitness Function</h1>

In [29]:
def fitness_function(agenda):
  id_voo = -1
  total_preco = 0
  for i in range(len(agenda) // 2):
    origem = pessoas[i][1]
    id_voo += 1
    ida = voos[(origem, destino)][agenda[id_voo]]
    total_preco += ida[2]
    id_voo += 1
    volta = voos[(destino, origem)][agenda[id_voo]]
    total_preco += volta[2]

  return total_preco

In [30]:
agenda = [1,7, 3,1, 1,1, 6,3, 2,4, 5,3]
fitness_function(agenda)

2619

In [32]:
!git clone https://github.com/gkhayes/mlrose.git
%pip install mlrose/.

Cloning into 'mlrose'...


Processing c:\users\alvin\documents\myprojects\pythonai\optimization_algorithms\mlrose
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/10.9 MB ? eta -:--:--
   ---------------------------------- ----- 9.4/10.9 MB 49.1 MB/s eta 0:00:01
   ---------------------------------------- 10.9/10.9 MB 29.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/44.5 MB ? eta -:--:--
   --------- ------------------------------ 10.2/44.5 MB 49.1 MB/s eta 0:00:01
   --------------------- ------------------ 24.4/44.5 MB 59.4 MB/s eta 0:00:01
   ----------------------------------- ---- 39.6/44.5 MB 64.6 MB/s eta 0:00:01
   ---------------------------------------  44.3/

In [34]:
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose

In [35]:
fitness = mlrose.CustomFitness(fitness_function)

In [36]:
problema = mlrose.DiscreteOpt(length=12, fitness_fn=fitness,
                              maximize = False, max_val = 10)

<h1>Hill Climb</h1>

In [39]:
melhor_solucao, melhor_custo = mlrose.hill_climb(problema, random_state = 1)
melhor_solucao, melhor_custo

(array([2, 6, 5, 7, 8, 1, 3, 6, 0, 2, 9, 1], dtype=int32), 1566.0)

In [40]:
imprimir_voos(melhor_solucao)

    Lisboa       LIS  9:15-12:03  99 15:07-17:21 129
    Madrid       MAD 14:22-16:32 126 17:06-20:00  95
     Paris       CDG 18:23-21:35 134  8:23-11:07 143
    Dublin       DUB 11:16-13:29  83 15:25-16:58  62
  Bruxelas       BRU  6:12-10:22 230  9:49-13:51 229
   Londres       LHR 20:30-23:11 114  8:19-11:16 122
Preço total:  1566


<h1>Simulated Annealing</h1>

In [63]:
melhor_solucao, melhor_custo = mlrose.simulated_annealing(problema,
                                                        #   schedule=mlrose.decay.GeomDecay(init_temp=10000), # ATUALIZAÇÃO FEV-2024
                                                          schedule=mlrose.GeomDecay(init_temp=10000), # ATUALIZAÇÃO FEV-2024
                                                          random_state=4)
melhor_solucao, melhor_custo

(array([2, 1, 5, 3, 9, 1, 9, 6, 0, 2, 9, 1], dtype=int32), 1668.0)

In [64]:
imprimir_voos(melhor_solucao)

    Lisboa       LIS  9:15-12:03  99  8:04-10:59 136
    Madrid       MAD 14:22-16:32 126 10:33-13:11 132
     Paris       CDG 19:53-22:21 173  8:23-11:07 143
    Dublin       DUB 20:17-22:22 102 15:25-16:58  62
  Bruxelas       BRU  6:12-10:22 230  9:49-13:51 229
   Londres       LHR 20:30-23:11 114  8:19-11:16 122
Preço total:  1668


<h1>Algoritmo Genético</h1>

In [80]:
melhor_solucao, melhor_custo = mlrose.genetic_alg(problema, pop_size=500, mutation_prob=0.2,
                                                  random_state=1)
melhor_solucao, melhor_custo

(array([7, 3, 1, 2, 9, 0, 3, 8, 0, 3, 4, 1]), 1956.0)

In [68]:
imprimir_voos(melhor_solucao)

    Lisboa       LIS 16:51-19:09 147 11:07-13:24 171
    Madrid       MAD  8:25-10:34 157  9:11-10:42 172
     Paris       CDG 19:53-22:21 173  6:33- 9:14 172
    Dublin       DUB 11:16-13:29  83 18:24-20:49 124
  Bruxelas       BRU  6:12-10:22 230 10:51-14:16 256
   Londres       LHR 12:08-14:59 149  8:19-11:16 122
Preço total:  1956
